    1- test function
    2- MC tests

In [ ]:
# ! delete the following
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

# 1- Test functions

In [ ]:
from mcrppy.integrand_test_functions import (support_integrands,
                                             f_1, f_2, f_3,
                                             exact_integral_f_1, exact_integral_f_2,
                                             exact_integral_f_3)


In [ ]:
nb_function=5
fct_list = [f_1, f_2, f_3]
fct_names = ["f_1", "f_2", "f_3"]
exact_integrals = [exact_integral_f_1, exact_integral_f_2, exact_integral_f_3]
for d in range(2,8):
    exact_integrals = [exact_integral_f_1(d), exact_integral_f_2(d), exact_integral_f_3(d)]
    print("for d={}".format(d))
    i_f = [{name: integ} for name, integ in zip(fct_names, exact_integrals)]
    print("Exact integral functions=", i_f )

In [ ]:
x = np.linspace(-3/5,3/5, 140)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_f_1 = f_1(points)
z_f_2 = f_2(points)
z_f_3 = f_3(points)
fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_1, c=z_f_1, rasterized=True)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_2, c=z_f_2, rasterized=True)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_3, c=z_f_3, rasterized=True)
ax.set_title(r"$f_3$")
plt.tight_layout()
#plt.savefig("linear_stat_used_for_var_comp.pdf")
plt.show()

# 2- MC tests 

In [ ]:
from mcrppy.monte_carlo_methods import mc_results, dataframe_error_test, dataframe_mse_results, dataframe_residual_test
from mcrppy.plot_functions import plot_mc_results, qq_plot_residual

In [ ]:
nb_samples=100
nb_cores = 3
nb_points_list=np.arange(50, 1050, 50).tolist()
fct_list = [f_1, f_2, f_3]
fct_names = ["f_1", "f_2", "f_3"]
estimators = ["MC", 
              "MCRB",
              "MCCV",
              "RQMC",
              "MCDPP"]
print("Number of tests: ", len(nb_points_list))
print("Number of points to be used:", nb_points_list)
print("Methods to be used:", estimators)

### For d=2

In [ ]:
d=2
exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d)]
support_window = support_integrands(d) 

In [ ]:
if __name__ == "__main__":
    results_2d, nb_points_2d = mc_results(d, 
                                          nb_points_list=nb_points_list, 
                                          nb_samples=nb_samples,
                                          support_window=support_window,
                                          fct_list=fct_list,
                                          fct_names=fct_names,
                                          exact_integrals=exact_integrals,
                                          estimators=estimators, 
                                          nb_cores=nb_cores,
                                          file_name="mc_results_2d_final.pickle"
                                            )

In [ ]:
plot_mc_results(d, 
                results_2d, 
                nb_points_2d, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False,
                estimators=estimators,
                save_fig="std_mc_2_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=results_2d, 
                        nb_points_list=nb_points_2d, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
qq_plot_residual(mc_list=results_2d, 
                 nb_points_list=nb_points_2d, 
                 fct_names=fct_names,
                #save_fig="qq_plot_2d.pdf"
                )

In [ ]:
plot_mc_results(d=2, 
                mc_list=results_2d, 
                nb_points_list=nb_points_2d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="Error",
                plot_error=True,
                plot_std=False,
                nb_subsample_nb_points=5,
                estimators=estimators,
                save_fig="error_mc_2d.pdf"
               )

In [ ]:
import matplotlib.pyplot as plt
mcrb = results_2d["MCRB"]['mc_results_f_3']['error_MCRB'][-1]
mc = results_2d["MC"]['mc_results_f_3']['error_MC'][-1]
mcdpp = results_2d["MCDPP"]['mc_results_f_3']['error_MCDPP'][-1]
rqmc = results_2d["RQMC"]['mc_results_f_3']['error_RQMC'][-1]
mccv = results_2d["MCCV"]['mc_results_f_3']['error_MCCV'][-1]
mc_results = [mc, mccv, mcrb, mcdpp, rqmc]
labels = ["MC","MCCV",  "MCBR", "MCDPP", "RQMC" ]
fix, ax = plt.subplots(figsize=(5,4))
ax.boxplot(x=mc_results, positions = [10, 20, 30, 40, 50], labels=labels)
ax.hlines(y=0, xmax=50, xmin=5, colors="grey", linestyles="--")
plt.show()

In [ ]:
# Square error
plot_mc_results(d=2, 
                mc_list=results_2d, 
                nb_points_list=nb_points_2d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="SE",
                plot_error=True,
                plot_std=False,
                log_scale=True,
                nb_subsample_nb_points=5,
                estimators=estimators,
                #save_fig="square_error_mc_2d.pdf"
               )

# 2- d=3

In [ ]:
d=3
nb_points_list=np.arange(50, 1050, 50).tolist()
exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d)]
support_window = support_integrands(d) 

In [ ]:
if __name__ == "__main__":
    results_3d, nb_points_3d = mc_results(d, 
                                          nb_points_list=nb_points_list, 
                                          nb_samples=nb_samples,
                                          support_window=support_window,
                                          fct_list=fct_list,
                                          fct_names=fct_names,
                                          exact_integrals=exact_integrals,
                                          estimators=estimators, 
                                          nb_cores=nb_cores,
                                          file_name="mc_results_3d_final.pickle"
                                            )

In [ ]:
plot_mc_results(d, 
                results_3d, 
                nb_points_3d, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False,
                estimators=estimators,
                save_fig="std_mc_3_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=results_3d, 
                        nb_points_list=nb_points_3d, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
plot_mc_results(d=d, 
                mc_list=results_3d, 
                nb_points_list=nb_points_3d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="Error",
                plot_error=True,
                plot_std=False,
                nb_subsample_nb_points=5,
                #estimators=["MC", "MCRB", "MCDPP"],
                #save_fig="square_error_mc_2d.pdf"
               )

In [ ]:
import matplotlib.pyplot as plt
mcrb = results_3d["MCRB"]['mc_results_f_3']['error_MCRB'][-1]
mc = results_3d["MC"]['mc_results_f_3']['error_MC'][-1]
mcdpp = results_3d["MCDPP"]['mc_results_f_3']['error_MCDPP'][-1]
rqmc = results_3d["RQMC"]['mc_results_f_3']['error_RQMC'][-1]
mccv = results_3d["MCCV"]['mc_results_f_3']['error_MCCV'][-1]
mc_results = [mc, mccv, mcrb, mcdpp, rqmc]
labels = ["uniform","MCCV",  "MCBR", "dpp", "RQMC" ]
fix, ax = plt.subplots(figsize=(5,4))
ax.boxplot(x=mc_results, positions = [10, 20, 30, 40, 50], labels=labels)
ax.hlines(y=0, xmax=50, xmin=5, colors="grey", linestyles="--")
plt.show()

# 3- d=4

In [ ]:
d=4
nb_points_list=np.arange(50, 1050, 50).tolist()
exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d)]
support_window = support_integrands(d)

In [ ]:
if __name__ == "__main__":
    results_4d, nb_points_4d = mc_results(d, 
                                          nb_points_list=nb_points_list, 
                                          nb_samples=nb_samples,
                                          support_window=support_window,
                                          fct_list=fct_list,
                                          fct_names=fct_names,
                                          exact_integrals=exact_integrals,
                                          estimators=estimators, 
                                          nb_cores=nb_cores,
                                          file_name="mc_results_4d_final.pickle"
                                            )

In [ ]:
plot_mc_results(d, 
                results_4d, 
                nb_points_4d, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False,
                estimators=estimators,
                save_fig="std_mc_4_d.pdf"
               )

In [ ]:
plot_mc_results(d=d, 
                mc_list=results_4d, 
                nb_points_list=nb_points_4d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="Error",
                plot_error=True,
                plot_std=False,
                nb_subsample_nb_points=5,
                #estimators=["MC", "MCRB", "MCDPP"],
                #save_fig="square_error_mc_2d.pdf"
               )

In [ ]:
import matplotlib.pyplot as plt
mcrb = results_4d["MCRB"]['mc_results_f_3']['error_MCRB'][-1]
mc = results_4d["MC"]['mc_results_f_3']['error_MC'][-1]
mcdpp = results_4d["MCDPP"]['mc_results_f_3']['error_MCDPP'][-1]
rqmc = results_4d["RQMC"]['mc_results_f_3']['error_RQMC'][-1]
mccv = results_4d["MCCV"]['mc_results_f_3']['error_MCCV'][-1]
mc_results = [mc, mccv, mcrb, mcdpp, rqmc]
labels = ["uniform","MCCV",  "MCBR", "dpp", "RQMC" ]
fix, ax = plt.subplots(figsize=(5,4))
ax.boxplot(x=mc_results, positions = [10, 20, 30, 40, 50], labels=labels)
ax.hlines(y=0, xmax=50, xmin=5, colors="grey", linestyles="--")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
mcrb = results_4d["MCP"]['mc_results_f_3']['error_MCP'][-1]
mc = results_4d["MC"]['mc_results_f_3']['error_MC'][-1]
#mcdpp = results_4d["MCDPP"]['mc_results_f_3']['error_MCDPP'][-1]
rqmc = results_4d["RQMC"]['mc_results_f_3']['error_RQMC'][-1]
mccv = results_4d["MCCV"]['mc_results_f_3']['error_MCCV'][-1]
mc_results = [mc, mccv, mcrb, rqmc]
labels = ["uniform","MCCV", "MCBR", "RQMC" ]
fix, ax = plt.subplots(figsize=(5,4))
ax.boxplot(x=mc_results,
           positions = [10, 20, 30, 40], 
           patch_artist=True,
           meanline=True,
           meanprops=dict(linestyle='--', linewidth=1.5, color='r'),
           showmeans=True,
           labels=labels)
ax.hlines(y=0, xmax=50, xmin=5, colors="grey", linestyles="--")
plt.show()